In [1]:
import pandas as pd

ts_data = pd.read_parquet('../data/transformed/ts_data_2022_01.parquet')

ts_data.head(5)

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1


In [7]:
ts_data[ts_data.pickup_location_id==43]

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,0,1
1,2022-01-01 01:00:00,0,1
2,2022-01-01 02:00:00,0,1
3,2022-01-01 03:00:00,0,1
4,2022-01-01 04:00:00,1,1
...,...,...,...
739,2022-01-31 19:00:00,0,1
740,2022-01-31 20:00:00,1,1
741,2022-01-31 21:00:00,1,1
742,2022-01-31 22:00:00,0,1


In [12]:
ts_data_one_location =  ts_data.loc[ts_data.pickup_location_id == 43,:].reset_index(drop = True)
ts_data_one_location.head(10)

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,97,43
1,2022-01-01 01:00:00,60,43
2,2022-01-01 02:00:00,22,43
3,2022-01-01 03:00:00,8,43
4,2022-01-01 04:00:00,6,43
5,2022-01-01 05:00:00,5,43
6,2022-01-01 06:00:00,3,43
7,2022-01-01 07:00:00,10,43
8,2022-01-01 08:00:00,7,43
9,2022-01-01 09:00:00,19,43


In [15]:
def get_cutoff_indices(data, n_features, step_size):
    stop_position = len(data) - 1

    #start the first sub-sequence at index position 0
    subseq_first_idx = 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features + 1
    indices = []

    while subseq_last_idx <= stop_position:
        indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))
        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size

    return indices

In [16]:
n_features=  24
step_size = 1
indices =  get_cutoff_indices(
    ts_data_one_location,
    n_features,
    step_size)

indices


[(0, 24, 25),
 (1, 25, 26),
 (2, 26, 27),
 (3, 27, 28),
 (4, 28, 29),
 (5, 29, 30),
 (6, 30, 31),
 (7, 31, 32),
 (8, 32, 33),
 (9, 33, 34),
 (10, 34, 35),
 (11, 35, 36),
 (12, 36, 37),
 (13, 37, 38),
 (14, 38, 39),
 (15, 39, 40),
 (16, 40, 41),
 (17, 41, 42),
 (18, 42, 43),
 (19, 43, 44),
 (20, 44, 45),
 (21, 45, 46),
 (22, 46, 47),
 (23, 47, 48),
 (24, 48, 49),
 (25, 49, 50),
 (26, 50, 51),
 (27, 51, 52),
 (28, 52, 53),
 (29, 53, 54),
 (30, 54, 55),
 (31, 55, 56),
 (32, 56, 57),
 (33, 57, 58),
 (34, 58, 59),
 (35, 59, 60),
 (36, 60, 61),
 (37, 61, 62),
 (38, 62, 63),
 (39, 63, 64),
 (40, 64, 65),
 (41, 65, 66),
 (42, 66, 67),
 (43, 67, 68),
 (44, 68, 69),
 (45, 69, 70),
 (46, 70, 71),
 (47, 71, 72),
 (48, 72, 73),
 (49, 73, 74),
 (50, 74, 75),
 (51, 75, 76),
 (52, 76, 77),
 (53, 77, 78),
 (54, 78, 79),
 (55, 79, 80),
 (56, 80, 81),
 (57, 81, 82),
 (58, 82, 83),
 (59, 83, 84),
 (60, 84, 85),
 (61, 85, 86),
 (62, 86, 87),
 (63, 87, 88),
 (64, 88, 89),
 (65, 89, 90),
 (66, 90, 91),
 (67,

In [20]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape = (n_examples,n_features), dtype = np.float32)
y = np.ndarray(shape = (n_examples), dtype = np.float32)

pickup_hours = []

for i, idx in enumerate(indices):
    x[i,:] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

In [21]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(719, 24)
x=array([[ 97.,  60.,  22., ...,  16.,  18.,   6.],
       [ 60.,  22.,   8., ...,  18.,   6.,   3.],
       [ 22.,   8.,   6., ...,   6.,   3.,   1.],
       ...,
       [ 28.,  16.,  13., ..., 102.,  66.,  61.],
       [ 16.,  13.,   8., ...,  66.,  61.,  73.],
       [ 13.,   8.,   1., ...,  61.,  73.,  33.]], dtype=float32)
pickup_hours[:5]=[Timestamp('2022-01-02 00:00:00'), Timestamp('2022-01-02 01:00:00'), Timestamp('2022-01-02 02:00:00'), Timestamp('2022-01-02 03:00:00'), Timestamp('2022-01-02 04:00:00')]


In [25]:
features_one_location = pd.DataFrame(
    x, 
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,97.0,60.0,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,...,70.0,94.0,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0
1,60.0,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,...,94.0,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0
2,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,...,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0
3,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,35.0,...,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0,1.0
4,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,35.0,77.0,...,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,52.0,36.0,28.0,16.0,13.0,8.0,1.0,1.0,2.0,1.0,...,78.0,74.0,66.0,91.0,117.0,100.0,106.0,147.0,121.0,102.0
715,36.0,28.0,16.0,13.0,8.0,1.0,1.0,2.0,1.0,1.0,...,74.0,66.0,91.0,117.0,100.0,106.0,147.0,121.0,102.0,66.0
716,28.0,16.0,13.0,8.0,1.0,1.0,2.0,1.0,1.0,4.0,...,66.0,91.0,117.0,100.0,106.0,147.0,121.0,102.0,66.0,61.0
717,16.0,13.0,8.0,1.0,1.0,2.0,1.0,1.0,4.0,9.0,...,91.0,117.0,100.0,106.0,147.0,121.0,102.0,66.0,61.0,73.0


In [26]:
targets_one_location = pd.DataFrame(
    y, 
    columns=[f'target_rides_next_hour']
)
targets_one_location

,target_rides_next_hour
0,3.0
1,1.0
2,1.0
3,0.0
4,0.0
...,...
714,66.0
715,61.0
716,73.0
717,33.0


In [35]:
from tqdm import tqdm

def transforme_ts_data_into_features_and_targets( ts_data, input_seq_len, step_size):
    assert set(ts_data.columns) =={'pickup_hour', 'rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets =  pd.DataFrame()

    for location_id in tqdm(location_ids):
        #keep only the time series data for this location
        ts_data_one_location = ts_data.loc[ts_data.pickup_location_id == location_id, ['pickup_hour','rides']]

        #precompute cutoff indices tosplit dataframe row
        indices = get_cutoff_indices(
            ts_data_one_location, input_seq_len, step_size
        )

        #slice and transpose data into numpy arrays for features and targets
        n_examples =  len(indices)
        x = np.ndarray(shape = (n_examples, input_seq_len), dtype = np.float32)
        y = np.ndarray(shape = (n_examples), dtype = np.float32)
        pickup_hours = []

        for i, idx in enumerate(indices):
            x[i,:] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        features_one_location = pd.DataFrame(
            x, 
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] =  pickup_hours
        features_one_location['pickup_location_id'] = location_id

        targets_one_location = pd.DataFrame(y,  columns=[f'target_rides_next_hour'])


        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace = True, drop = True)
    targets.reset_index(inplace = True, drop = True)

    return features, targets['target_rides_next_hour']

        

        

        



In [37]:
features, targets = transforme_ts_data_into_features_and_targets(
    ts_data, 24*7*1, 24
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 257/257 [00:03<00:00, 65.74it/s]

features.shape=(6168, 170)
targets.shape=(6168,)
